### Set-ip

In [1]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(usecwd=True),override=True) # read local .env file

from sqlalchemy import create_engine
from os import environ

username     =  environ.get("POSTGRES_USERNAME", "postgres")
password     =  environ.get("POSTGRES_PASSWORD", "postgres")
ipaddress    =  environ.get("POSTGRES_IPADDRESS", "localhost")
port         =  environ.get("POSTGRES_PORT", "5432")
dbname       =  environ.get("POSTGRES_DBNAME", "ArlingtonMA")

cnx= create_engine(f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}')

In [2]:
FINANCIALS_VISIBLE_TRACES = [
    'Education','Police','Fire','Public Works',
    'Average Tax Bill as a % of Income',
    'SP',
    'CIP % of Total Value',
    'Water & Sewer','WaterSewer',
    'EquilizedValues',
    'Total Tax Levy','Levy Ceiling','Override Capacity',
    'Automobiles','Light Trucks','Total Vehicles',
    'Overlay Reserve Amount',
    'Condominiums 102','Two Family 104',
    'ResidentBirths',
    'Total Miles',
    'Stabilization Fund as % of Budget',
    'RO Levy as % of Total',
    'RO% of Total',
    'MBTA','Total Assessments',
    'enterprise_fund_Water_revenues',
    'Actual','Estimate',
    'fedgrants_Total_Revenues','fedgrants_Total Expenditures',
    'health_exp',
    'Unemployment Rate',
    'meals','rooms',
    'Total Outstanding Debt',
    'Total New Growth Value','Residential New Growth Value',
    'Total Employees',
    'population',
    'State Aid as % of Total','Local Receipts as % of Total',
    'stategrants_Education_revenues',
    'OPEB Trust Funds_revenues','OPEB Trust Funds_expenditures'
]

In [3]:
from collections import defaultdict

def ddict():
    return defaultdict(ddict)

def ddict2dict(d):
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = ddict2dict(v)
    return dict(d)

all_params = ddict()

###    TABLES

#### PROPERTY ##

In [4]:
## DEEDS ##

all_params['property']['deeds']['tables']['detail'] = {
    'title' : """
                ***{ppt} - {dataset} ***  
                {count} Documents for {date} totaling {total} 
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
    """,
    'query'   : """
                    SELECT   date, 
                             concat(s."streetnum", ' ' , s."streetname", ' ' ,s.unit) as address,
                             name[1],deed_type,consideration,
                             book::int, page::int ,docno,pages,
                             pid
                    FROM     property.deeds_details s
                    WHERE    extract(year from s.date)::int = {year}
                    ORDER BY date DESC, book DESC, page DESC;
                """,
    'query_dates' : """
                        SELECT distinct(cast(date_part('year', date) as int)) as dates 
                        FROM property.deeds_details
                        ORDER BY dates;
                    """,  
}


all_params['property']['deeds']['tables']['summary'] = {
    'title' : """
                ***{ppt} - {dataset} ***  
                Count of all documents from summary and detail extracts.
                [src](http://www.masslandrecords.com/MiddlesexSouth)
    
    """,
    'query'   : """
               SELECT   extract(year from s.date)::int as year, 
                        'summary' as extract_type,
                        count(distinct deed_type) as deed_types,
                         count(*) as count,NULL as consideration
                FROM     property.deeds_summary s
                WHERE town =10
                GROUP BY extract(year from s.date)::int
                UNION
                SELECT extract(year from d.date)::int as year, 
                        'detail' as extract_type,
                         count(distinct deed_type) as deed_types,
                         count(*) as count,
                         sum(consideration) as consideration
                FROM property.deeds_details d
                GROUP BY extract(year from date)::int,extract_type
                ORDER BY year desc, extract_type;
                """,
}

all_params['property']['deeds']['tables']['summary by year and filing type'] = {
    'title' : """
                ***{ppt} - {dataset} ***    
                Count of Documents for {date} across {count} document types totaling {total}.
                [src](http://www.masslandrecords.com/MiddlesexSouth)    
    """,
    'query'   : """
               SELECT   extract(year from s.date)::int as year, 
                        'summary' as extract_type,
                        deed_type,
                         count(*) as count,NULL as consideration
                FROM     property.deeds_summary s
                WHERE town =10 and extract(year from s.date)::int = {year}
                GROUP BY extract(year from s.date)::int,extract_type,deed_type
                UNION
                SELECT extract(year from d.date)::int as year, 
                        'detail' as extract_type,
                         deed_type,
                         count(*) as count,
                         sum(consideration) as consideration
                FROM property.deeds_details d
                WHERE extract(year from d.date)::int = {year}
                GROUP BY extract(year from date)::int,extract_type,deed_type
                ORDER BY year desc, count desc, extract_type;
                """,
    'query_dates' : """
                        SELECT distinct(cast(date_part('year', date) as int)) as dates 
                        FROM property.deeds_details
                        ORDER BY dates;
                    """,
}


all_params['property']['deeds']['tables']['sales summary'] = {
    'title' : """
                ***{ppt} - {dataset} ***    
                Summary of Sales derived from Deed transfers by year and land use.
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
    
    """,
    'query' : """
                    SELECT date_part('year',date) as year,land_use,
                    		count(*) as count, 
                    		sum(consideration) as total,
                    		avg(consideration)::int as average, 
                    		min(consideration) as minimum, 
                    		max(consideration) as maximum 
                    FROM property.deeds_details d
                    LEFT JOIN property.assessments a on a.pid = d.pid and a.year=date_part('year',date)
                    WHERE deed_type=96
                    and consideration>1000
                    GROUP BY 1,2
                    ORDER BY year desc, land_use;
                """,
}

In [5]:
## Energy ##

all_params['property']['energy']['tables']['BNL detail'] = {
    'title' : """
                ***Energy - Berkeley National Labs ***    
                Details on {count} solar panel installations 
                [src](https://emp.lbl.gov/tracking-the-sun)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/energy/solar%20BNL.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/energy/solar%20BNL%20ETL.ipynb)                
    """,
    'query':"""
                select 
                    date,
                    "system_size_DC" as "kW", 
                    total_installed_price as cost, 
                    rebate_or_grant as grant,
                    bnl_installer_name,
                    COALESCE(module_quantity_1,0)+COALESCE(module_quantity_2,0)+COALESCE(module_quantity_3,0) as panels,
                    nameplate_capacity_module_1 as panel_watts,
                    bnl_module_manufacturer_1 as bnl_module_manufacturer,
                    bnl_module_model_1 as bnl_module_model,
                    efficiency_module_1 as efficiency,
                    "dateOfBatteryInstall",
                    "battery_rated_capacity_kW" as "battery_kw",
                    "battery_rated_capacity_kWh" as "battery_kwh",
                    bnl_battery_manufacturer,
                    bnl_battery_model
                from energy.solar_bnl 
                where dor=10
                order by date desc;
            """,
}

all_params['property']['energy']['tables']['EV chargers'] = {
    'title' : """
                ***Energy - EV Chargers ***                    
                {count} Electric Vehicle charger install permits issued totalling {total}
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    
    """,
    'query' : """
                    SELECT 
                        date, address,
                        permit_value, permit, 
                        owner, contractor, 
                        description
                    FROM   property.permits p
                    WHERE  upper(description) similar to upper('%%((charging)|(charger))%%')
                    OR     description similar to ('%%((EV ))%%')
                    ORDER BY date desc
                    ;
                """,
    
}

all_params['property']['energy']['tables']['PTS detail'] = {
    'title' : """
                ***Solar Energy - Production Tracking System ***   
                Details of {count} solar panel installations. 
                [src](https://www.masscec.com/sites/default/files/documents/PVinPTSwebsite%202022-02-22.xlsx)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/energy/solar%20PTS.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/energy/solar%20PTS%20ETL.ipynb)                
    """,
    'query':"""
                select date,round(watt/1000.0,3) as "kW",
                "est_annual_kWh",
                pts_installer,
                pts_module_mfgr,
                pts_inverter_mfgr,
                pts_meter_mfgr,
                pts_utility,
                pts_owner,
                pts_srec,
                pts_program,
                pts_type
                from energy.solar_pts 
                where dor=10
                order by date desc;
            """,
}

all_params['property']['energy']['tables']['PTS summary'] = {
    'title' : """
                ***Solar Energy - Production Tracking System ***   
                Summary of solar panel installations across {count} years
                [src](https://www.masscec-pts.com/#/home)
    """,
    'query':"""
                select year,count(*) as installs,
                round(sum(watt)/1e3,0) as "kW",round(sum("cost"),0) as "cost",
                round(sum("grant"),0) as "grants",
                round(sum("est_annual_kWh")/1e3) as "MWh"
                from energy.solar_pts 
                where dor=10
                group by year
                order by year desc;
            """,
}

all_params['property']['energy']['tables']['PTS summary all'] = {
    'title' : """
                ***Solar Energy - Production Tracking System ***   
                Summary of solar panel installations across {count} years for Massachusetts
                [src](https://www.masscec-pts.com/#/home)
    """,
    'query':"""
                select year,count(distinct(dor)) as towns,count(*) as installs,
                round(sum(watt)/1e6,0) as "MW",round(sum("cost")/1e6,0) as "cost $MM",
                round(sum("grant")/1000,0) as "grants $1000s",
                round(sum("est_annual_kWh")/1e6) as "GWh"
                from energy.solar_pts 
                where dor<=352
                group by year
                order by year desc;
            """,
}

all_params['property']['energy']['tables']['heat pumps'] = {
    'title' : """
                ***Heat Pumps - Clean Energy Center ***  
                {count} Heat Pumps installed thru 2019 totaling {total}
                [src](https://files-cdn.masscec.com/uploads/Residential%20ASHP%20Data_For%20Website%20%282%29.xlsx)
    
    """,
    'query'   : """
                select  installed as date,
                        address,
                        total,rebate,
                        capacity,
                        fuel,backup,
                        occupants,footage,ac,
                        heating_costs,outdoor_units,indoor_units,
                        installer,manufacturer
                from property.heatpumps 
                order by installed desc;
                """,
    
}


all_params['property']['energy']['tables']['solar arrays'] = {
    'title' : """
                ***Solar Energy - Solar Array Inventory ***  
                Details on {count} solar panel installations costing {total} with {cats}MW capacity
                [1](https://seigniorage.medium.com/mapping-the-solar-system-61ba53fd23aa)
                [2](https://emp.lbl.gov/tracking-the-sun) 
                [3](https://www.masscec.com/sites/default/files/documents/PVinPTSwebsite%202022-02-22.xlsx)
                [4](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    """,
    'query' : """
                select 
                    date,
                    concat(z."streetnum", ' ' , z."streetname", ' ' ,z.unit) as address,
                    z.owner,
                	s.pid,
                	watt,
					modules.panels,
					inverters.quantity as inverters,
					modules.bnl_efficiency,
                	watts->>'est_annual_kwh' as est_annual_kwh,
                	cost,
                	dates->>'last_permit_date' as permit_date,
                	financials->>'total_permit_values' AS total_permit_values ,
                	(entity->>'rps_type')::INTEGER as rps_type,
                	entity->>'name' as name,
                	financials->>'permit_owner' as permit_owner,
                	financials->>'contractor' as contractor,
                	(entity->>'pts_program')::INTEGER as pts_program,
                	(financials->>'rps_installer')::INTEGER as rps_installer,
                	financials->>'rebate_or_grant' as rebate_or_grant,
                	(financials->>'pts_srec')::INTEGER as pts_srec,
                	financials->>'permit' as permits,
                	financials->>'permit_description'::VARCHAR as permit_description,
                	(watts->>'pts_module_mfgr')::INTEGER as pts_module_mfgr,
                	(watts->>'pts_inverter_mfgr')::INTEGER as pts_inverter_mfgr,
                	(amps->>'pts_meter_mfgr')::INTEGER as pts_meter_mfgr
                FROM 
                	energy.solar s
				CROSS  JOIN LATERAL (
				   SELECT 
						sum((COALESCE(obj ->> 'quantity'))::int) AS panels,
						100.0*AVG((obj ->> 'efficiency')::REAL) AS bnl_efficiency
				   FROM   JSONB_ARRAY_ELEMENTS(watts->'modules') obj
				   ) modules
				CROSS  JOIN LATERAL (
				   SELECT 
						sum((COALESCE(obj ->> 'quantity'))::int) AS quantity
				   FROM   JSONB_ARRAY_ELEMENTS(watts->'inverters') obj
				   ) inverters
                LEFT JOIN 
                	property.assessments z on z.year = 2023 and z.pid = s.pid
                ORDER BY date DESC;
            """,
    
}

all_params['property']['energy']['tables']['solar contractors'] = {
    'title' : """
                ***Solar Energy - Production Tracking System ***   
                Leader board of {count} solar contactors since 2019
                [src](https://www.masscec-pts.com/#/home)
    """,
    'query':"""
                select 
                	i.value as installer, 
                	count(*) as installs,
                	round(AVG(cost/(watt*1.0)),2) as cost_per_watt,
                	round(sum(watt/1000.0),3) as total_kw_installed, 
                	sum(cost) as total_cost,
                	percentile_cont(0.5) WITHIN GROUP (ORDER BY watt/1000.0) as median_kw_installed
                from energy.solar_pts p
                left join common.int_value_pairs t 
                	on t.item='pts_type' and t.key = p.pts_type
                left join common.int_value_pairs i 
                	on i.item='pts_installer' and i.key = pts_installer
                where dor=10
                and date>='2019-01-01' 
                and t.value like '%%Residential%%'
                and cost>0
                and cost/(watt*1.0)<10.0
                group by i.value
                order by total_kw_installed desc
            """,
}

all_params['property']['energy']['tables']['solar contractors - all'] = {
    'title' : """
                ***Solar Energy - Production Tracking System ***   
                Leader board of {count} solar contactors for Massachusetts since 2020
                [src](https://www.masscec-pts.com/#/home)
    """,
    'query':"""
                select 
                	i.value as installer, 
                	count(*) as installs,
                	round(AVG(cost/(watt*1.0)),2) as cost_per_watt,
                	round(sum(watt/1000.0),3) as total_kw_installed, 
                	sum(cost) as total_cost,
                	percentile_cont(0.5) WITHIN GROUP (ORDER BY watt/1000.0) as median_kw_installed
                from energy.solar_pts p
                left join common.int_value_pairs t 
                	on t.item='pts_type' and t.key = p.pts_type
                left join common.int_value_pairs i 
                	on i.item='pts_installer' and i.key = pts_installer
                where date>='2020-01-01' 
                and t.value like '%%Residential%%'
                and cost>0
                and cost/(watt*1.0)<10.0
                group by i.value
                order by total_kw_installed desc
            """,
}


all_params['property']['energy']['tables']['systems'] = {
    'title' : """
                ***Household Energy Systems ***  
                {count} Building heating and cooling systems from Assessors
                [src](https://www.arlingtonma.gov/departments/finance/assessor)
    
    """,
    'query' : """
            SELECT pid,address,
            round((interior->>'Heat Fuel')::numeric,0)::int as "Heat Fuel",
            round((interior->>'Heat Type')::numeric,0)::int as "Heat Type",
            interior->>'Air Conditioned' as "Air Conditioned", 
            interior->>'Solar Hot Water' as "Solar Hot Water",
            interior->>'Percent Heated' as "Percent Heated", 
            round((interior->>'Electric')::numeric,0)::int as "Electric",
            round((interior->>'Insulation')::numeric,0)::int as "Insulation"
    
            FROM property.patriot p
            ;
    """,
    #round((interior->>'Heating Systems')::numeric,0)::int as "Heating Systems",
    
}

all_params['property']['energy']['tables']['solar production'] = {
    'title' : """
                ***Solar Production ***  
                {count} Arrays
                [src]()[1][2]
    
    """,
    'query' : """
                select sp.pid, a.address, 
                	a.owner->>'owner2' as owner, 
                	to_char(timestamp,'YYYY') as year,
                	sum(kwh) as kwh
                from energy.solar_production sp
                left join property.patriot a on  a.pid=sp.pid
                group by 1,2,3,4
                order by 1,4;
            """,   
}


In [6]:
## PARCELS ##

all_params['property']['parcels']['tables']['detail'] = {
    'title' : """
                ***{ppt} - {dataset} from MassGIS***   
                {count} Tax Parcels in {date} assessed for {total}
                [src](https://www.mass.gov/info-details/massgis-data-property-tax-parcels)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels%20ETL.ipynb)
    
    """,
    'query' : """
                SELECT concat(streetnum,' ',streetname, case when unit = '' then '' else ' #'||unit end) as address, 
                    total as assessed, owner, land_use, style, 
                    area as lot_size, land, living_area, building, 
                    units, rooms 
                FROM   property.assessments 
                WHERE  year = {year} 
                ORDER BY year, streetname, streetnum, unit;
                """,
    'query_dates' : """SELECT distinct(year) as dates from property.assessments order by dates;""",
    
}
                            # sum(area)/43600.0 as acres

all_params['property']['parcels']['tables']['summary'] = {
    'title' : """
                ***{ppt} - {dataset} from MassGIS***   
                Summary of assessed values by year and land use.
                [src](https://www.mass.gov/info-details/massgis-data-property-tax-parcels)    
    """,
    'query' : """
                    SELECT year,land_use,
                            count(*) as count, 
                            sum(total) as total,
                            sum(land) as land,
                            sum(building) as building
                    FROM property.assessments 
                    GROUP BY year, land_use
                    ORDER BY count desc, year desc;
                """,
}


all_params['property']['parcels']['tables']['patriot'] = {
    'title' : """
                ***{ppt} - {dataset} from Patriot Properties***   
                {count} Tax Parcels for {date} totaling {total}
                [src](https://arlington.patriotproperties.com/default.asp)
    """,
    'query' : """
                SELECT  p.address,
                        p.owner->>'owner1' as "Owner",
                        pa.land_use,
                        exterior->>'Year Built' as year_built,
                        p.zoning,cards,
                        p."lotSize",
                        assessments->>'buildingValue' as "building",
                        assessments->>'yardValue' as "yard",
                        assessments->>'landValue' as "land",
                        assessments->>'totalValue' as "assessed",
                        image
                from property.patriot p
                left join property.assessments pa on pa.pid=p.pid and pa.year=p.year
                WHERE   p.year = {year}
                order by "streetname","streetnum"
                ;
            """,
    'query_dates' : """SELECT distinct(year) as dates from property.patriot order by dates;""",
    
}


In [7]:
## PERMITS ##

all_params['property']['permits']['tables']['detail'] = {
    'title' : """
                ***{ppt} - {dataset} ***   
                {count} Permits for {date} totalling {total}
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits%20ETL.ipynb)
    
    """,
    'query' : """
                select date, address,
                       permit_value, permit_fee,
                       description,
                       owner, contractor,
                       permit_type,permit, permit_class
                from property.permits 
                where year = {year} 
                order by date DESC, permit;
            """,
    'query_dates' : """SELECT distinct(year) as dates from property.permits order by dates;""",
    
}
all_params['property']['permits']['tables']['summary'] = {
    'title' : """
                ***{ppt} - {dataset} ***   
                Summary of Permits across {count} years and 4 Permit types
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    """,
    'query' : """
                    SELECT year,permit_type,
                            count(*) as count, 
                            sum(permit_fee) as fees,
                            sum(permit_value) as total,
                            avg(permit_value) as average, 
                            min(permit_value) as minimum, 
                            max(permit_value) as maximum 
                    FROM property.permits 
                    WHERE permit_value>0
                    GROUP BY year, permit_type
                    ORDER BY permit_type, year desc;
                """,
}

In [8]:
## SALES ##

all_params['property']['sales']['tables']['Recent'] = {
    'title' : """
                ***Real Estate Property Sales - Registry of Deeds***   
                {count} Recent Sales totalling {total}
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
    
    """,
    'query' : """
                select p.pid,p.image,p.address,
                	a.land_use,
                	a.style,
                	a."area" as lot_size,
                	a."living_area" as living_area,
                	p.rooms->>'Rooms' as rooms,
                    p.rooms->>'Bedrooms' as bedrooms,
                    p.interior->>'Full Baths' as bathrooms,
                    p.interior->>'1/2 Bath' as halfbath,
                	d.date,d.consideration as price, d.name,d.grant_type
                from property.deeds_details d
                left join property.patriot p on p.pid = d.pid
                left join property.assessments a on a.pid = d.pid and a.year=date_part('year',d.date)
                where consideration>100
                and deed_type=96
                order by date desc, timestamp desc                
                LIMIT 200
                ;
            """,    
}

all_params['property']['sales']['tables']['detail'] = {
    'title' : """
                ***Real Estate Property Sales - Assessors***   
                {count} Sales for {date} totalling {total}
                [src](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales%20ETL.ipynb)
    
    """,
    'query' : """
                    SELECT date,
                            concat( "streetNum",' ',"streetName", ' ', "unit" ) as address,
                            price,
                            buyer,
                            seller,
                            sale_type,
                            land_use
                    FROM property.sales 
                    WHERE year = {year}
                    ORDER BY date;
                """,
    'query_dates' : """SELECT distinct(year) as dates from property.sales order by dates;""",
    
}


all_params['property']['sales']['tables']['summary'] = {
    'title' : """
                ***Real Estate Property Sales - Assessors***   
                Summary of Sales by land use and year
                [src](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)
    
    """,
    'query' : """
                    SELECT year,land_use,
                            count(*) as count, 
                            sum(price) as total,
                            avg(price) as average, 
                            min(price) as minimum, 
                            max(price) as maximum 
                    FROM property.sales 
                    GROUP BY year, land_use
                    ORDER BY land_use, year desc;
                """,
}

#### PEOPLE ##

In [9]:
## Elections ##

all_params['people']['elections']['tables']['detail'] = {
    'traces' : [
        'Local',
        'Mid-term',
        'Presidential',
        'State Primary',
        'Special State',
        'Presidential Primary',
        'Override'],
    'title' : """
                ***Elections***   
                {count} Voters for the {date} election
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    """,
    'query':"""
                SELECT  people_id 
                FROM    people.elections 
                WHERE   '{date}' = ANY( date ) 
                ;
            """,
    'query_dates' : """
                SELECT    date  AS dates 
                FROM      common.election_dates 
                WHERE     value='{item}' 
                ORDER BY  dates;
            """,
}

all_params['people']['elections']['tables']['summary'] = {
    'title' : """
                ***Elections***   
                Count of voters across {count} elections by election type and date.
    
    """,
    'query':"""
            SELECT     ed.value as election_type, 
                        b.date as date,count(*) as count
            FROM       people.elections as a, UNNEST(date) as b
            LEFT JOIN common.election_dates ed on ed.date=b.date
            GROUP BY   b.date, ed.value
            ORDER BY   ed.value,b.date desc;
        """,
}

In [10]:
## Registered ##

all_params['people']['registered']['tables']['detail'] = {
    'title' : """
                ***People - Voter Rolls***   
                {count} Registered Voters on {date}
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://www.mass.gov/info-details/mass-general-laws-c51-ss-47c)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    
    """,
    'query':"""
                SELECT  people_id 
                FROM    people.registered 
                WHERE   '{date}' = ANY( date ) 
                ;
            """,
    'query_dates' : """
                SELECT DISTINCT(UNNEST(date)) AS dates 
                FROM      people.registered 
                ORDER BY  dates;
            """,

}

all_params['people']['registered']['tables']['summary'] = {
    'title' : """
                ***People - Voter Rolls***   
                Count of registered voters over {count} observations.
    
    """,
    'source'  : """[source](https://www.mass.gov/info-details/mass-general-laws-c51-ss-47c)""",
    'discuss' : """""",
    'query':"""
            SELECT     b.date as date,count(*) as count
            FROM       people.registered as a, UNNEST(date) as b
            GROUP BY   b.date
            ORDER BY   b.date desc;
        """,
}


In [11]:
## Residents ##

all_params['people']['residents']['tables']['detail'] = {
    'title' : """
                ***People - Residents***   
                {count} Residents in the {date} True List
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://malegislature.gov/Laws/GeneralLaws/PartI/TitleVIII/Chapter51/Section6)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
                [4](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
    """,
    'query':"""
                SELECT  people_id 
                FROM    people.residents 
                WHERE   '{date}' = ANY( date ) 
                ;
            """,
    'query_dates' : """
                SELECT DISTINCT(UNNEST(date)) AS dates 
                FROM      people.residents 
                ORDER BY  dates;
            """,

}

all_params['people']['residents']['tables']['summary'] = {
    'title' : """
                ***People - Residents***   
                Count of residents over {count} recordings.
                [1](https://malegislature.gov/Laws/GeneralLaws/PartI/TitleVIII/Chapter51/Section6)
    """,
    'query':"""
            SELECT     b.date as date,count(*) as count
            FROM       people.residents as a, UNNEST(date) as b
            GROUP BY   b.date
            ORDER BY   b.date desc;
        """,
}

#### GOVERNANCE ####

In [12]:
## checkbook

all_params['governance']['financials']['tables']['checkbook'] = {
    'title' : """
                ***Financials - Open Checkbook***   
                Detail of {count} checks totalling {total} for FY{date}
                [src](https://www.arlingtonma.gov/departments/finance/financial-budget-information/open-checkbook)
    """,
    'query':"""
                select 
                    date,amount,checkbook_department,checkbook_vendor,checkbook_description,
                    checkbook_department_category,checkbook_fund_type,check_number
                from governance.checkbook 
                where dor=10
                and year={year}
                order by date desc, check_number desc;
            """,
    'query_dates' : """SELECT distinct(year) as dates from governance.checkbook order by dates;""",

}

In [13]:
## DOR

all_params['governance']['financials']['tables']['DOR'] = {
    'title' : """
                ***Financials - Department of Revenue Databank***   
                {total} Series under {series} over {count} years
                [src](https://www.mass.gov/lists/schedule-a-reports-revenues-expenditures-and-more)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/governance/dor_databank.md)
    """,
    'query'   : """
                    select d.year,d.value,d.zscore,d.dor_databank_series 
                    from governance.dor_databank d
                    where  d.dor in (10,999)
                    AND  d.dor_databank_series in (
                        select ivp.key
                        from common.int_value_pairs ivp
                        LEFT JOIN common.int_value_pairs ivp2
                            ON ivp2.item = 'dor_databank_series_types' 
                            AND ivp2.value='{series_type}'
                        LEFT JOIN common.int_value_pairs ivp3
                            ON ivp3.item = 'dor_databank_series_type' 
                            AND ivp3.value=ivp2.value
                        WHERE ivp.item='dor_databank_series'
                        and ivp.key=ivp3.key
                        and ivp3.key is not NULL
                    )
                    ORDER BY  d.year DESC
                    ;
                """,
    'query_series' : """SELECT key,value FROM common.int_value_pairs WHERE item='dor_databank_series_types';""",
}

In [14]:
## prop2.5 ##
all_params['governance']['financials']['tables']['prop 2.5'] = {
    'title' : """
                ***Financials - Proposition 2.5***    
                Property Tax Overrides and Debt Exclusions by expenditure year
                [src](https://www.mass.gov/lists/schedule-a-reports-revenues-expenditures-and-more)
                [1](https://www.mass.gov/service-details/proposition-2-12-and-tax-rate-process)
                
    
    """,
    'query'   : """
                    select * 
                    from governance.prop25_votes
                    where  dor = 10
                    ORDER BY  year DESC
                    ;
                """,
}

In [15]:
## TMM ##
all_params['governance']['meetings']['tables']['town meeting members'] = {
    'title' : """
                ***Governance - HOA***    
                Town Meeting Members for {date}
                [src](https://www.arlingtonma.gov/home/showpublisheddocument/64717/638163762463230000)
    
    """,
    'query' : """
                SELECT
                    *
                FROM governance.tmm
                WHERE year={year}
                ;
            """,
    'query_dates' : """
                    SELECT distinct(year) as dates 
                    FROM governance.tmm 
                    ORDER BY dates;
                """,
}

all_params['governance']['meetings']['tables']['transcripts'] = {
    'title' : """
                ***Board and Committee Meetings***   
                {metric} of {trace} for {date}
                [src](https://acmi.tv/programs/arlington-government-channel/)
    
    """,
    'traces': ['select-board-meetings','school-committee','zoning-board-of-appeals','redevelopment-board-meetings','town-meeting'],
    'metrics':['raw','outline','Q&A'],
    'query' : """
                SELECT transcript as raw,markdown as outline, qa as "Q&A"
                FROM governance.meetings m
                LEFT JOIN common.int_value_pairs ivp on ivp.item='authority' and ivp.key=m.authority
                WHERE dor = 10
                    AND date='{date}'
                    AND ivp.value='{item}'
                ;
            """,
    'query_dates' : """
                    SELECT distinct(date) as dates 
                    FROM governance.meetings m
                    LEFT JOIN common.int_value_pairs ivp on ivp.item='authority' and ivp.key=m.authority
                    WHERE ivp.value='{item}'
                    ORDER BY dates;
                """,
    

}

#### Infrastructure ####

In [16]:
## Business ##

all_params['infrastructure']['business']['tables']['ppp loans'] = {
    'title' : """
            ***Businesses receiving Paycheck Protection Program loans***   
            {count} loans totalling {total}
            [src](https://data.sba.gov/dataset/ppp-foia)
    
    """,
    'query' : """
                    SELECT "BorrowerName" as business,
                            "DateApproved" as date,
                            "BorrowerAddress" as address,
                            "CurrentApprovalAmount" as total,
                            "ForgivenessAmount" as forgiven,
                            "JobsReported" as employees,
                            "PAYROLL_PROCEED" as "payroll",
                            "UTILITIES_PROCEED" as "utilities",
                            "RENT_PROCEED" as "rent",
                            "HEALTH_CARE_PROCEED" as "health care",
                            "MORTGAGE_INTEREST_PROCEED" as "mortgage",
                            "Race","Ethnicity","Veteran","NonProfit",
                            "NAICSCode" as naics,
                            n.value as industry,
                            "InitialApprovalAmount" as initial,
                            "LoanNumber" as "loan number",
                            "FranchiseName" as "franchise",
                            "BusinessAgeDescription" as "age",
                            address as "alt address"
                    FROM infrastructure.ppp p
                    LEFT JOIN common.naics n on n.key=p."NAICSCode"
                    ORDER BY "CurrentApprovalAmount" DESC
                    ;
                """,
    
}

all_params['infrastructure']['grid']['tables']['circuits'] = {
    'title' : """
                ***Eversource Electric Grid***   
                {total} Transformers on {count} Circuits 
                [src](https://eversource.maps.arcgis.com/apps/webappviewer/index.html?id=7b13d31f908243e49406f198b359aa71)
    
    """,
    'query' : """
                SELECT
                    circuit_id, amps, 
                    bulk_substation as "substation", 
                    bulk_substation_voltage as "volts",
                    "bulk_substation_rating_MVA" as "bulk MVA",
                    substation_type as "type", 
                    "substation_rating_MVA" as "MVA",
                    substation as "substation*", 
                    sections
                FROM infrastructure.grid
                ;
            """

}

all_params['infrastructure']['roads']['tables']['inventory'] = {
    'title' : """
                ***Roads and Intersections***   
                Inventory of {count} roads with {total} intersections
                [src](https://gis-arlingtonma.opendata.arcgis.com/maps/ed93de524bbe48f1a65e454cdbf7ef97)
                [1](https://www.arlingtonma.gov/home/showdocument?id=2468)
    
    """,
    'query' : """
            SELECT "streetName", from_street, to_street, class, segments
            FROM   infrastructure.roads
            ORDER BY class, from_street, to_street
            ;
    """,

}


In [17]:
## SCHOOLS ##

all_params['infrastructure']['schools']['tables']['summary'] = {
    'traces' : [
        'ppx',
        'mcas',
        'enrollment',
        'teacher_age',
        'teacher_race',
        'teacher_salaries',
        'teacher_program_area'],
    'title' : """
                ***Schools - Enrollment, Expenditures, Performance and Teachers***   
                {trace} - {total} metrics across {count} observations
                [src](https://profiles.doe.mass.edu/statereport)
    
    """,
    'query':"""
                select x.*
                from infrastructure.schools_{item} x
                left join common.school_id_location lo 
                        on lo.school_id=x.school_id
                where lo.dor = 10
            """,
}


all_params['infrastructure']['schools']['tables']['types'] = {
    'title' : """
                ***School Enrollment***   
                By school type and year
                [src](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx)
    
    """,
    'query' :"""
                select schools_type,year,value as students 
                from infrastructure.schools_types
                where dor=10
                order by schools_type, year desc
                ;
            """,
}

all_params['infrastructure']['schools']['tables']['types by town'] = {
    'title' : """
                ***School Enrtollment***   
                {date} Enrollment in {trace} for all towns
                [src](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx)
    
    """,
    'traces':[
        "Local Public Schools","Academic Regional Schools",
        "Vocational Technical Regional Schools",
        "Collaboratives","Charter Schools",
        "Out-of-District Public Schools","Home Schooled",
        "In State Private and Parochial Schools",
        "Out-of-State Private and Parochial Schools",
        "Total","Total Public"
    ],
    'query' :"""
                select s.dor,s.value as students 
                from infrastructure.schools_types s
                left join common.int_value_pairs ivp 
                    on ivp.key=s.schools_type and ivp.item='schools_type'
                where s.year={year}
                and ivp.value='{item}'
                order by schools_type, dor
                ;
            """,
    'query_dates' : """SELECT distinct(year) as dates from infrastructure.schools_types order by dates;""",
}

In [18]:
## WATER ##


all_params['infrastructure']['water']['tables']['bills'] = {
    'title' : """
                ***Water - Bills and Usage***   
                {count} Bills for {date} totalling {total}
                [src](https://www.arlingtonma.gov/departments/public-works)
    
    """,
    'query'  :  """
                   SELECT 
                       a.address, b.amount, b.usage,
                       a.owner, b.account
                   FROM infrastructure.water_bills b
                   LEFT JOIN infrastructure.water_accounts a ON a.account = b.account
                    WHERE b.date = '{date}'
                    ORDER BY b.account
                   ;
                """,
    'query_dates' : """
                        SELECT distinct(date) as dates 
                        FROM infrastructure.water_bills 
                        ORDER BY dates;""",


}


all_params['infrastructure']['water']['tables']['summary'] = {
    'title' : """
                ***Water - Bills and Usage***   
                Total usage and charges over {count} quarters
                [src](https://www.arlingtonma.gov/departments/public-works)
    
    """,
    'query'  :  """
                    select date,count(*) as count,
                            sum(amount)::int as total,
                            sum(usage) as usage,
                            ROUND((sum(amount)/count(*))::numeric,2) as amount
                    from infrastructure.water_bills
                    group by date
                    order by date desc                   ;
                """,


}

### MAPS ###

In [19]:
#### PEOPLE ##

In [20]:
## ELECTIONS
all_params['people']['elections']['maps']['scatter']={
    'title' : """
                ***Elections***   
                {count} Voters for the {date} election
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    """,
    'color' : 'precinct',
    'zoom'  : 12.5,
    'traces' : [
        'Local',
        'Mid-term',
        'Presidential',
        'State Primary',
        'Special State',
        'Presidential Primary',
        'Override'],
    'query':"""
                SELECT  people_id 
                FROM    people.elections 
                WHERE   '{date}' = ANY( date ) 
                ;
            """,
    'query_dates' : """
                SELECT    date  AS dates 
                FROM      common.election_dates 
                WHERE     value='{item}' 
                ORDER BY  dates;
            """,
}

In [21]:
## REGISTERED
all_params['people']['registered']['maps']['scatter']={
    'title' : """
                ***People - Voter Rolls***   
                {count} Registered Voters on {date}
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://www.mass.gov/info-details/mass-general-laws-c51-ss-47c)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    
    """,
    'color' : 'precinct',
    'zoom'  : 12.5,
    'query':"""
                SELECT  people_id 
                FROM    people.registered 
                WHERE   '{date}' = ANY( date ) 
                ;
            """,
    'query_dates' : """
                SELECT DISTINCT(UNNEST(date)) AS dates 
                FROM      people.registered 
                ORDER BY  dates;
            """,
    
}

In [22]:
## RESIDENTS
all_params['people']['residents']['maps']['scatter']={
    'title' : """
                ***People - Residents***   
                {count} Residents in the {date} True List
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://malegislature.gov/Laws/GeneralLaws/PartI/TitleVIII/Chapter51/Section6)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
                [4](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
    """,
    'color' : 'precinct',
    'zoom'  : 12.5,
    'query':"""
                SELECT  people_id 
                FROM    people.residents 
                WHERE   '{date}' = ANY( date ) 
                ;
            """,
    'query_dates' : """
                SELECT DISTINCT(UNNEST(date)) AS dates 
                FROM      people.residents 
                ORDER BY  dates;
            """,
    
}

In [23]:
#### PROPERTY ####

In [24]:
## DEEDS ##

all_params['property']['deeds']['maps']['detail'] = {
    'title' : """
                ***Registry of Deeds***  
                {count} Documents for {date} totaling {total} 
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
        """,
    'color' : 'deed_type',
    'traces':['deed_type'],
    'traces_visible' : ["MORTGAGE","DEED"],
    'metrics':['consideration'],
    'zoom'  : 12.5,
    'query' : """
                SELECT extract(year from s.date)::int as year,
                date, book::int, page::int ,deed_type,docno,
                consideration,
                concat(s."streetnum", ' ' , s."streetname", ' ' ,s.unit) as address,
                grant_type, name, pid
                FROM property.deeds_details s
                WHERE extract(year from s.date)::int = {date};
            """,
    'query_dates' : """
            SELECT distinct(extract(year from date)::int) as dates 
            from property.deeds_details 
            order by dates;
        """,
}

all_params['property']['deeds']['maps']['sales'] = {
    'title' : """
                ***Registry of Deeds***  
                {count} Sales for {date} totaling {total} 
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
        """,
    'color' : 'deed_type',
    'traces':['deed_type'],
    'traces_visible' : ["MORTGAGE","DEED"],
    'metrics':['consideration'],
    'zoom'  : 12.5,
    'query' : """
                SELECT extract(year from s.date)::int as year,
                date, book::int, page::int ,deed_type,docno,
                consideration,
                concat(s."streetnum", ' ' , s."streetname", ' ' ,s.unit) as address,
                grant_type, name, pid
                from property.deeds_details s
                WHERE 
                    consideration>1000
                AND
                    extract(year from s.date)::int = {date};
    """,
    'query_dates' : """
                        SELECT distinct(extract(year from date)::int) as dates 
                        from property.deeds_details 
                        order by dates;
                    """,
}

In [25]:
## ENERGY ##
all_params['property']['energy']['maps'][' Solar systems'] = {
    'title' : """
                ***Solar Systems***     
                Detail of {count} solar panel installations costing {total} with {cats}MW capacity.
                [src](https://emp.lbl.gov/tracking-the-sun) 
                [1](https://seigniorage.medium.com/mapping-the-solar-system-61ba53fd23aa)
                [2](https://www.masscec.com/sites/default/files/documents/PVinPTSwebsite%202022-02-22.xlsx)
    """,
    'map_type':'mapbox_choropleth',
    'color':'watt',
    'traces':['watt'],
    'range_color':[2000,10000],
    'geometry_key':'struct_id',
    'zoom'  : 12.5,
    'query' : """
                select ss.struct_id,pp.address,pp.owner->>'owner1' as owner,
    			pp.image,modules.panels as panels, s.*,ss.geometry
                from energy.solar s 
                left join common.structures ss on ss.pid = s.pid 
    				and ss."area"=(select max(z."area") from common.structures z where z.pid=s.pid)
                left join property.patriot pp on pp.pid = s.pid
    				CROSS  JOIN LATERAL (
    				   SELECT 
    						sum((COALESCE(obj ->> 'quantity'))::int) AS panels
    				   FROM   JSONB_ARRAY_ELEMENTS(watts->'modules') obj
    				   ) modules
                where s.pid is not null 
    			ORDER BY struct_id
               ;
            """,
}



all_params['property']['energy']['maps']['EV chargers'] = {
    'title' : """
                ***Energy - EV Chargers ***                    
                {count} Electric Vehicle charger install permits issued totalling {total}
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    
    """,
    'color' : 'year',
    'traces':['year','contractor'],
    'traces_visible' : ['2022','2021'],
    'zoom'  : 12.5,
    'query' : """
                    SELECT p.year::varchar, date, permit_type, permit, address, 
                            owner, contractor, description, permit_value, p.pid
                    FROM   property.permits p
                    WHERE  upper(description) similar to upper('%%((charging)|(charger))%%')
                    OR     description similar to ('%%((EV ))%%')
                    ;
                """,
}


all_params['property']['energy']['maps']['Heat Pumps'] = {
    'title' : """
                ***Heat Pumps - Clean Energy Center ***   
                {count} Heat Pumps installed thru 2019 totaling {total}
                [src](https://files-cdn.masscec.com/uploads/Residential%20ASHP%20Data_For%20Website%20%282%29.xlsx)
    
    """,
    'color' : 'year',
    'traces':['year','installer','manufacturer','fuel'],
    'traces_visible' : ['2015','2016','2017','2018','2019'],
    'zoom'  : 12.5,
    'query' : """
                select date_part('year', installed)::varchar as year,
                hp.*
                from property.heatpumps hp
                order by installed desc;
                """,
    
}


all_params['property']['energy']['maps']['Heat Pumps 2'] = {
    'title' : """
                ***Heat Pumps - Building Permits ***    
                {count} Heat Pumps installed from Permits totalling {total}
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    
    """,
    'color' : 'year',
    'traces':['year','contractor'],
    'traces_visible' : ['2022'],
    'zoom'  : 12.5,
    'query' : """
                SELECT p.year::varchar, date, permit, address, 
                        owner, contractor, description, permit_value, p.pid
                FROM   property.permits p
                WHERE  upper(description) similar to upper('%%((heat pump))%%')
                ORDER  BY date
                ;
                """,
}




all_params['property']['energy']['maps']['Solar - Groups'] = {
    'title' : """
                ***Solar System Installers and Manufacturer***    
                Detail of {count} solar panel installations costing {total} with {cats}MW capacity.
                [src](https://emp.lbl.gov/tracking-the-sun)
                [1](https://seigniorage.medium.com/mapping-the-solar-system-61ba53fd23aa)
                [2](https://www.masscec.com/sites/default/files/documents/PVinPTSwebsite%202022-02-22.xlsx)
    """,
    'color' : 'year',
    'traces':['year','rps_installer','pts_module_mfgr','pts_inverter_mfgr'],
    'traces_visible' : ['2022','2021','2020','2016','Enphase Energy','Fronius USA',
                        'SMA America','SolarEdge Technologies',
                        'SunPower','Yaskawa Solectria Solar',
                        'Trinity Solar','SunRun','SunBug Solar',
                        'SolarFlair Energy, Inc',
                        'SolarCity Corporation DBA Telsa Energy',
                        'Ameresco, Inc.','Great Sky Solar'
                       ],
    'zoom'  : 12.5,
    'query' : """
                select 
                    pp.address,
                	date_part('year',date)::CHAR(4) as year,
                	s.pid,
                	date,
                	watt,
					modules.panels,
					inverters.quantity as inverters,
					modules.bnl_efficiency,
                	watts->>'est_annual_kwh' as est_annual_kwh,
                	cost,
                	dates->>'last_permit_date' as permit_date,
                	financials->>'total_permit_values' AS total_permit_values ,
                	entity->>'owner' as owner,
                	(entity->>'rps_type')::INTEGER as rps_type,
                	entity->>'name' as name,
                	financials->>'permit_owner' as permit_owner,
                	financials->>'contractor' as contractor,
                	(entity->>'pts_program')::INTEGER as pts_program,
                	(financials->>'rps_installer')::INTEGER as rps_installer,
                	financials->>'rebate_or_grant' as rebate_or_grant,
                	(financials->>'pts_srec')::INTEGER as pts_srec,
                	financials->>'permit' as permits,
                	financials->>'permit_description'::VARCHAR as permit_description,
                	(watts->>'pts_module_mfgr')::INTEGER as pts_module_mfgr,
                	(watts->>'pts_inverter_mfgr')::INTEGER as pts_inverter_mfgr,
                	(amps->>'pts_meter_mfgr')::INTEGER as pts_meter_mfgr
                FROM 
                	energy.solar s
                 LEFT JOIN property.patriot pp on pp.pid = s.pid
				CROSS  JOIN LATERAL (
				   SELECT 
						sum((COALESCE(obj ->> 'quantity'))::int) AS panels,
						100.0*AVG((obj ->> 'efficiency')::REAL) AS bnl_efficiency
				   FROM   JSONB_ARRAY_ELEMENTS(watts->'modules') obj
				   ) modules
				CROSS  JOIN LATERAL (
				   SELECT 
						sum((COALESCE(obj ->> 'quantity'))::int) AS quantity
				   FROM   JSONB_ARRAY_ELEMENTS(watts->'inverters') obj
				   ) inverters
                ;
                """,
}

all_params['property']['energy']['maps']['HVAC'] = {
    'title' : """
                ***Household Energy Systems ***   
                {count} Building heating and cooling systems from Assessors
                [src](https://www.arlingtonma.gov/departments/finance/assessor)
    
            """,
    'color' : "Heat Fuel",
    'traces':["Solar Hot Water", "Air Conditioned", "Heat Fuel", "Heat Type"],
    'traces_visible' : ['Yes','100','Gas','Elec Base/B'],
    'zoom'  : 12.5,
    'query' : """
                    SELECT p.pid,address,
                    case when cast((interior->>'Solar Hot Water') as INTEGER)=0 then 'No' else 'Yes' end as "Solar Hot Water",
                    cast(case when (interior->>'Air Conditioned')='' then '0' else (interior->>'Air Conditioned') end as float)::INTEGER as "Air Conditioned",
                    cast((interior->>'Heat Fuel') as float)::INTEGER as "Heat Fuel", 
                    cast((interior->>'Heat Type') as float)::INTEGER as "Heat Type" 
                    FROM property.patriot p
                    ;
                """,    
}


In [26]:
## parcels ##
                # SELECT cast(10*date_part('decade',to_date(year_built::varchar, 'yyyy')) as int)::text as decade_built,
                #         a.pid,a.land_use, a.style,a.zoning
                # FROM   property.assessments a
                # WHERE  year = {year}
    # 'map_type':'mapbox_choropleth',
    # 'range_color':[200000,2000000],
    # 'geometry_key':'loc_id',

all_params['property']['parcels']['maps']['MassGIS'] = {
    'title' : """
                ***{ppt} - {dataset} from MassGIS***   
                {count} Tax Parcels in {date} assessed for {total}
                [src](https://www.mass.gov/info-details/massgis-data-property-tax-parcels)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels%20ETL.ipynb)
    
    """,
    'color' : 'land_use',
    'traces' : ['land_use','style','zoning','decade_built'] ,
    'traces_visible' : ["104 - Two Family","R0","Cape","Colonial","1920"],
    'zoom'  : 12.5,
    'query' : """
                SELECT cast(10*date_part('decade',to_date(year_built::varchar, 'yyyy')) as int)::text as decade_built,
                        a.*
                FROM   property.assessments a
                WHERE  year = {year}
            """,
    'query_dates' : """SELECT distinct(year) as dates from property.assessments order by dates;""",
    
}

# all_params['property']['parcels']['maps']['Assessor'] = {
#     'title' : """
#                 ***{ppt} - {dataset} from Assessor***   
#                 {count} Tax Parcels in {date} assessed for {total}
#                 [src](https://arlington.patriotproperties.com/default.asp)
#                 [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels.md)
#                 [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels%20ETL.ipynb)
    
#     """,
#     'color' : 'land_use',
#     'traces':['land_use','year_built'],
#     'traces_visible' : ['101 - Single Family','1920'],
#     'zoom'  : 12.5,
#     'query' : """
#                 select p.pid,p.year,p.address,pa.land_use,
#                     exterior->>'Year Built' as year_built,
#                     p.rooms->>'Rooms' as rooms,
#                     p.rooms->>'Bedrooms' as bedrooms
#                 from property.patriot p
#                 left join property.assessments pa on pa.pid=p.pid and pa.year=p.year
#                 ;
#                 """,
#     'query_dates' : """SELECT distinct(year) as dates from property.patriot order by dates;""",
    
# }

In [27]:
## PERMITS ##
all_params['property']['permits']['maps']['detail'] = {
    'title' : """
                ***{ppt} - {dataset} ***   
                {count} Permits for {date} totalling {total}
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits%20ETL.ipynb)
    
    """,
    'color' : 'permit_type',
    'traces' : ['permit_type'],
    'traces_visible' : ["Residential","Building"],
    'zoom'  : 12.5,
    'query' : """
                    select p.*
                    from property.permits p 
                    where p.year = {year}
                    ;
                """,
    'query_dates' : """SELECT distinct(year) as dates from property.permits order by dates;""",    
}


In [28]:
## SALES ##
all_params['property']['sales']['maps']['scatter'] = {
    'title' : """
                ***Real Estate Property Sales - Assessors***   
                {count} Sales for {date} totalling {total}
                [src](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales%20ETL.ipynb)
    
    """,
    'color' : 'land_use',
    'traces':['land_use','sale_type'],
    'traces_visible' : ["101 - Single Family","Arms Length"],
    'zoom'  : 12.5,
    'query' : """
                    SELECT year,date,
                            concat( "streetNum",' ',"streetName", ' ', "unit" ) as address,
                            price,
                            buyer,
                            seller,
                            sale_type,
                            land_use,pid
                    FROM property.sales 
                    WHERE year = {year}
                """,
    'query_dates' : """SELECT distinct(year) as dates from property.sales order by dates;""",
}


In [29]:
#### INFRASTRUCTURE ####


all_params['infrastructure']['grid']['maps']['line']={
    'title' : """
                ***Eversource Electric Grid***   
                {count} line sections 
                [src](https://eversource.maps.arcgis.com/apps/webappviewer/index.html?id=7b13d31f908243e49406f198b359aa71)
    
    """,
    'map_type':'mapbox_line',
    'color' : 'circuit_id',
    'traces':['circuit_id'],
    'traces_visible' : [
        "211-H11","211-H14","211-H3",
        "320-H7","351-1385H1","59-1383H",
        "59-1383H1","59-1393H","59-1393H1","59-1471",
        "5901","5903","5904","5905","5907",
        "826-25","826-28",
        "828-1382","828-1NA9","828-1NC5",
        "828-1NC8","828-1ND3","TRM-PTC"
    ],
    'zoom'  : 12.5,
    'query' : 'select * from infrastructure.grid_segments;',
}

all_params['infrastructure']['roads']['maps']['line'] = {
    'title' : """
                ***Roads and Intersections***   
                Inventory of {count} roads segments
                [src](https://gis-arlingtonma.opendata.arcgis.com/maps/ed93de524bbe48f1a65e454cdbf7ef97)
                [1](https://www.arlingtonma.gov/home/showdocument?id=2468)
    
    """,
    'map_type':'mapbox_line',
    'color' : 'class',
    'traces':['class'],
    'traces_visible' : ["Local Street","Private Way"],
    'zoom'  : 12.5,
    'query' : 'select * from infrastructure.road_segments;',    
}

In [30]:
## WATER ##
all_params['infrastructure']['water']['maps']['parcels'] = {
    'title' : """
                ***Infrastructure***   
                {count} water bills and usage for {date}
                [src](https://www.arlingtonma.gov/departments/public-works)
    """,
    'map_type':'mapbox_choropleth',
    'color' : 'amount',
    'traces' : ['amount','usage'] ,
    'traces_visible' : ["2022-08-31"],
    'zoom'  : 12.5,   
    'geometry_key':'loc_id',
    'range_color_amount':[0,1500],
    'range_color_usage':[0,100],
    'query'  :  """SELECT 
                        b.date,a.address, b.amount, b.usage,
                        a.owner, b.account, a.pid
                    FROM infrastructure.water_bills b
                    LEFT JOIN infrastructure.water_accounts a ON a.account = b.account
                    WHERE b.date = '{date}'
                    ORDER BY b.account
                    ;
                """,
    'query_dates' : """
                    SELECT distinct(date) as dates 
                    FROM infrastructure.water_bills 
                    ORDER BY dates;
                """,
}


In [31]:
## BUSINESS ##

all_params['infrastructure']['business']['maps']['ppp scatter'] = {
    'title' : """
            ***Business - Paycheck Protection Program***   
             {count} loans totalling {total}
             [src](https://data.sba.gov/dataset/ppp-foia)
    
    """,
    'color' : "BusinessType",
    'traces' : ['BusinessType','industry'],
    'zoom'  : 12.5,
    'query' : """
                    SELECT p.pid,
                            "BusinessType",
                            "BorrowerName" as business,
                            "DateApproved" as date,
                            "BorrowerAddress" as address,
                            "CurrentApprovalAmount" as total,
                            "ForgivenessAmount" as forgiven,
                            "JobsReported" as employees,
                            "PAYROLL_PROCEED" as "payroll",
                            "UTILITIES_PROCEED" as "utilities",
                            "RENT_PROCEED" as "rent",
                            "HEALTH_CARE_PROCEED" as "health care",
                            "MORTGAGE_INTEREST_PROCEED" as "mortgage",
                            "Race","Ethnicity","Veteran","NonProfit",
                            "NAICSCode" as naics,
                            n.value as industry,
                            address as "alt address"
                    FROM infrastructure.ppp p
                    LEFT JOIN common.naics n on n.key=p."NAICSCode"
                    ;
                """,    
    'traces_visible' : ['Sole Proprietorship','Corporation',
                        'Child Care Services','Landscaping Services',
                        'Offices of Real Estate Agents and Brokers'],
}

In [32]:
#### GOVERNANCE ####

all_params['governance']['meetings']['maps']['scatter']={
    'title' : """
                ***Governance***    
                {count} Town Meeting Members during {date}
                [src](https://www.arlingtonma.gov/home/showpublisheddocument/64717/638163762463230000)
    
    """,
    'traces' : ['precinct'],
    'color' : 'precinct',
    'zoom'  : 12.5,
    'query' : """
                SELECT
                    *
                FROM governance.tmm
                WHERE year={year}
                ;
            """,
    'query_dates' : """
                    SELECT distinct(year) as dates 
                    FROM governance.tmm 
                    ORDER BY dates;
                """,
}



all_params['governance']['financials']['maps']['state']={
    'title' : """
                ***Financials - Town Comparisons***   
                {series_metric} for {date}
                [src](https://www.mass.gov/lists/schedule-a-reports-revenues-expenditures-and-more)
    
    """,
    'map_type':'mapbox_choropleth',
    'color'   : 'zscore',
    'traces'   : ['zscore'],
    'lat'   :  42,
    'lon'   : -71.3,
    'zoom'  : 7.6,
    'geometry_key':'municipality',
    'query'   : """
                    SELECT          p1.value as municipality, p2.value as series,
                    				f.value as value,f.zscore,
                    				ST_Simplify( tb.geometry::geometry, 0.0005 ) as geometry
                    FROM            governance.dor_databank f 
                    LEFT JOIN       common.int_value_pairs p2 on p2.key = f.dor_databank_series and p2.item='dor_databank_series' 
                    LEFT JOIN       common.int_value_pairs p3 on p3.value = '{series_metric}' and p3.item='dor_databank_series' 
                    LEFT JOIN       common.int_value_pairs p1 on p1.key = f.dor and p1.item='dor' 
                    LEFT JOIN       common.town_boundaries tb on tb.town_id = f.dor  
                    WHERE           f.year={year}
                    AND             f.dor_databank_series=p3.key
                    ;
                """,
    'query_series' : """SELECT key,value FROM common.int_value_pairs WHERE item='dor_databank_series_types';""",
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   governance.dor_databank
                        WHERE  year >= 1990
                        AND    year <= 2023
                        ORDER BY dates;
                    """,
}


### CHARTS #

#### LINE ###

##### PEOPLE ####

In [33]:
## people
##people stand alone, not by attribute!
##in {groups} {trace} groups

all_params['people']['elections']['charts']['line'] = {
    'title' : """
                ***{ppt} - {dataset}***   
                Recorded activity by date 
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    """,
    'chart_type':'line',
    'XY'      : ( 'date' , 'count' )                ,
    'color'   : 'record_type'                            ,
    'traces'   : ['record_type']                            ,
    'traces_visible'  : [ 'Local', 'Registered Voters', 'Residents' ] ,
    'query'   : """
                    SELECT     ed.value as record_type, b.date,count(*) 
                    FROM       people.elections as a, UNNEST(date) as b
                    LEFT JOIN  common.election_dates ed on ed.date = b.date
                    GROUP BY   ed.value, b.date

                    UNION

                    SELECT     'Registered Voters' as record_type, b.date,count(*) 
                    FROM       people.registered as a, UNNEST(date) as b
                    GROUP BY   1, b.date

                    UNION

                    SELECT     'Residents' as record_type, b.date,count(*) 
                    FROM       people.residents as a, UNNEST(date) as b
                    GROUP BY   1, b.date

                    ORDER BY   1,2
                    ;
                """,
}

all_params['people']['registered']['charts']['line'] = all_params['people']['elections']['charts']['line']
all_params['people']['residents']['charts']['line']  = all_params['people']['elections']['charts']['line']


##### PROPERTY ####

In [34]:
## DEEDS ##
all_params['property']['deeds']['charts']['line'] = {
    'title' : """
                ***REGISTERY OF DEEDS***    
                {metric} by year and {trace}
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
    """,
    'chart_type':'line',
    'metrics' : ['consideration','count'],
    'XY'      : ( 'year' , 'item' ),
    'traces'   : ['deed_type'],
    'color'   : 'deed_type' ,
    'traces_visible'  : [ 'DEED', 'MORTGAGE' ] ,
    'query'   : """
                    SELECT     cast(date_part('year', date) as int) as year, deed_type , 
                               count(*) as count, sum(consideration) as consideration 
                    FROM       property.deeds_details
                    GROUP BY   date_part('year', date), deed_type
                    ORDER BY   year, deed_type;
                """,
    
}

In [35]:
## parcels

all_params['property']['parcels']['charts']['line'] = {
    'title' : """
                ***{ppt} - {dataset} from MassGIS***    
                {metric} assessments by year in {groups} {trace} groups
                [src](https://www.mass.gov/info-details/massgis-data-property-tax-parcels)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels%20ETL.ipynb)
    
    """,
    'chart_type':'line',
    'metrics' : [ 'total' , 'building' , 'land', 'total_chg_1y', 'land_chg_1y' ]     ,
    'XY'      : ( 'year' , 'item' )                ,
    'color'   : 'land_use'                            ,
    'traces'   : ['land_use']            ,
    'traces_visible'  : [ '000 - Total', '101 - Single Family' ] ,
    'query'   : """
                    SELECT     cast(date_part('year',to_date(year::varchar, 'yyyy')) as int) as year,
                               land_use , count(*) as parcels, 
                               sum(land) as land, sum(building) as building, sum(total) as total,
                               ROUND(100*cast(avg(total_chg_1y) as numeric),2) as total_chg_1y,
                               round(100*cast(avg(land_chg_1y) as numeric),2) as land_chg_1y
                    FROM       property.assessments
                    GROUP BY   year, land_use
                    UNION
                    SELECT     cast(date_part('year',to_date(year::varchar, 'yyyy')) as int) as year,
                               999 as land_use, count(*) as parcels, 
                               sum(land) as land, sum(building) as building, sum(total) as total,
                               round(100*cast(avg(total_chg_1y) as numeric),2) as total_chg_1y,
                               round(100*cast(avg(land_chg_1y) as numeric),2) as land_chg_1y
                    FROM       property.assessments
                    GROUP BY   year
                    ORDER BY   year, land_use;
                """,
}


In [36]:
## Energy
all_params['property']['energy']['charts']['solar line'] = {
    'title' : """
                ***Solar Energy***   
                Solar {metric} over {count} years
                [1](https://seigniorage.medium.com/mapping-the-solar-system-61ba53fd23aa)
                [2](https://emp.lbl.gov/tracking-the-sun) 
                [3](https://www.masscec.com/sites/default/files/documents/PVinPTSwebsite%202022-02-22.xlsx)
                [4](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    """,
    'chart_type':'line',
    'color'   : ['region'],
    'traces'   : ['region']                            ,
    'traces_visible'  : [ 'local' ] ,    
    'metrics' : ['installs','total','cost_per_watt','mw_installed'] ,
    'XY'      : ( 'year' , 'item' ) ,
    'query':"""
                select 'local' as region,cast(date_part('year',date) as int) as year,
                        count(*) as installs,
                        sum(cost) as "total",
                        round(AVG(cost/(watt*1.0)),2) as cost_per_watt,
                    	round(sum(watt/1e6),2) as mw_installed 
                from energy.solar 
                WHERE dor=10
                GROUP BY 1, 2

                UNION
                
                select 'state' as region, cast(date_part('year',date) as int) as year,
                        count(*) as installs,
                        sum(cost) as "total",
                        round(AVG(cost/(watt*1.0)),2) as cost_per_watt,
                    	round(sum(watt/1e6),2) as mw_installed 
                from energy.solar_pts 
                GROUP BY 1,2
                
                ORDER by 1,2;
            """,

}

# all_params['property']['energy']['charts']['utilities'] = {
#     'title' : """
#                 ***Utilities***   
#                 {metric} over {count} months
#                 [1]()
#     """,
#     'chart_type':'line',
#     'color'   : ['service'],
#     'traces'   : ['service']                            ,
#     'traces_visible'  : [ 'gas','electric' ] ,    
#     'metrics' : ['value','supply','delivery','total','daily_usage','daily_cost'] ,
#     'XY'      : ( 'date' , 'item' ) ,
#     'query': """
#                 select pid,'electric' as service,
#                 		date, kwh as value,
#                 		round((kwh*generation)::numeric,2) as supply,
#                 		round((kwh*delivery+customer_charge)::numeric,2) as delivery,
#                 		round((kwh*generation + kwh*delivery+customer_charge)::numeric,2) as total,
#                 		round((kwh/days)::numeric,1) as daily_usage,
#                 		round(((kwh*generation + kwh*delivery+customer_charge)/days)::numeric,2) as daily_cost
#                 from energy.electric_utility
                
#                 UNION
#                 select pid,'gas' as service,
#                 		date, therms as value,
#                 		round((therms*cost_per_therm)::numeric,2) as supply,
#                 		round((days*min_charge + therms*(tier1_delivery+tier1_del_adj))::numeric,2) as delivery,
#                 		round((therms*cost_per_therm + days*min_charge + therms*(tier1_delivery+tier1_del_adj))::numeric,2) as total,
#                 		round((therms/days)::numeric,1) as daily_usage,
#                 		round(((therms*cost_per_therm + days*min_charge + therms*(tier1_delivery+tier1_del_adj))/days)::numeric,2) as daily_cost
#                 from energy.gas_utility
#                 order by pid, service,date
#                 ;
#             """,
# }

In [37]:
## PERMITS ##
all_params['property']['permits']['charts']['line'] = {
    'title' : """
                ***{ppt} - {dataset} ***   
                {metric} by year in {groups} {trace} groups
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits%20ETL.ipynb)
    
    """,
    'chart_type':'line',
    'metrics' : ['permit_value','permit_fee', 'counts' ]     ,
    'XY'      : ( 'year' , 'item' )                ,
    'traces'   : ['permit_type']                            ,
    'color'   : 'permit_type'                            ,
    'traces_visible'  : [ 'Building' ] ,
    'query'   : """
                    SELECT     year, permit_type , count(*) as counts, 
                               sum(permit_value) as permit_value, sum(permit_fee) as permit_fee 
                    FROM       property.permits
                    GROUP BY   year, permit_type
                    ORDER BY   year, permit_type;
                """,
    
}


In [38]:
## SALES ##
all_params['property']['sales']['charts']['line'] = {
    'title' : """
                ***Real Estate Property Sales - Assessors***   
                {metric} by year in {groups} {trace} groups
                [src](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales%20ETL.ipynb)
    
    """,
    'chart_type':'line',
    'metrics' : ['amount','count']     ,
    'XY'      : ( 'year' , 'item' )                ,
    'color'   : 'land_use'                            ,
    'traces'   : ['land_use']                            ,
    'traces_visible'  : [ '101 - Single Family' ] ,
    'query'   :   """
                    SELECT     year, land_use , count(*) as count, 
                               sum(price) as amount 
                    FROM       property.sales
                    GROUP BY   year, land_use
                    ORDER BY   year, land_use;
                  """,
    
}


##### GOVERNANCE ####

In [39]:
## FINANCIALS ##
all_params['governance']['financials']['charts']['checkbook'] = {
    'title' : """
                ***Financials - Open Checkbook***   
                Summary of Checkbook activity by {trace} - all dates
                [src](https://www.arlingtonma.gov/departments/finance/financial-budget-information/open-checkbook)
    """,
    'chart_type':'line',
    'metrics' : ['total','checks'],
    'color'   : ['checkbook_department'],
    'traces'  : ['checkbook_department','checkbook_department_category','checkbook_fund_type'],
    'XY'      : ( 'year' , 'item' ),
    'query':"""
                select year,{item},
                    sum(amount) as total,count(*) as checks
                from governance.checkbook 
                where dor=10
                group by year,{item}
                order by year;
            """,
    'traces_visible'     : ['EDUCATION','TOWN MANAGER','SNOW & ICE',
                            'HIGHWAY','LIBRARY','INFORMATION TECHNOLOGY',
                           'ENTERPRISE FUNDS'],

}

all_params['governance']['financials']['charts']['monthly checkbook'] = {
    'title' : """
                ***Financials - Open Checkbook***   
                Monthly Summary of Checkbook by {trace}
                [src](https://www.arlingtonma.gov/departments/finance/financial-budget-information/open-checkbook)
    """,
    'chart_type':'line',
    'metrics' : ['total','checks'],
    'color'   : ['checkbook_department'],
    'traces' : ['checkbook_department','checkbook_department_category','checkbook_fund_type'],
    'XY'      : ( 'date' , 'item' ),
    'query':"""
            	select 
            		to_char(date, 'YYYY-MM') as date,
            		{item},
            	sum(amount) as total,count(*) as checks
            	from governance.checkbook 
            	where dor=10
            	group by 1,{item}
            	order by {item}, date;
             """,
    'traces_visible'     : ['EDUCATION','TOWN MANAGER','SNOW & ICE',
                            'HIGHWAY','LIBRARY','INFORMATION TECHNOLOGY',
                           'ENTERPRISE FUNDS'],

}



all_params['governance']['financials']['charts']['DOR  line'] = {
    'title' : """
                ***Financials - Department of Revenue Databank***   
                {groups} Series under {series} - all years
                [src](https://www.mass.gov/lists/schedule-a-reports-revenues-expenditures-and-more)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/governance/dor_databank.md)
    """,
    'chart_type':'line',
    'metrics' : ['value','zscore'],
    'XY'      : ( 'year' , 'item' ),
    'color'   : 'dor_databank_series',
    'traces'   : ['dor_databank_series'],
    'query'   : """
                    SELECT           f.year,f.value as value,f.zscore, f.dor_databank_series
                    FROM             governance.dor_databank f 
                    WHERE            f.dor=10 
                    AND  f.dor_databank_series in (
                        select ivp.key
                        from common.int_value_pairs ivp
                        LEFT JOIN common.int_value_pairs ivp2
                            ON ivp2.item = 'dor_databank_series_types' 
                            AND ivp2.value='{series_type}'
                        LEFT JOIN common.int_value_pairs ivp3
                            ON ivp3.item = 'dor_databank_series_type' 
                            AND ivp3.value=ivp2.value
                        WHERE ivp.item='dor_databank_series'
                        and ivp.key=ivp3.key
                        and ivp3.key is not NULL
                    )
                    ORDER BY dor_databank_series, year
                    ;
                """,
    'traces_visible'  : FINANCIALS_VISIBLE_TRACES,
}




##### Infrastructure ####

In [40]:
## WATER ##
all_params['infrastructure']['water']['charts']['line'] = {
    'title' : """
                ***Infrastructure***   
                Water {metric} by billing date
                [src](https://www.arlingtonma.gov/departments/public-works)
    """,
    'chart_type':'line',
    'color':['water'],
    'traces':['water'],
    'traces_visible':['water'],
    'metrics' : ['usage','amount','accounts']    ,
    'XY'      : ( 'date' , 'item' )                ,
    'query'   : """
                    select 'water' as water,date,
                    count(*) as accounts,
                    sum(amount)::int as amount,
                    sum(usage) as usage,
                    ROUND((sum(amount)/count(*))::numeric,2) as avg
                    from infrastructure.water_bills
                    group by 1,date
                    order by 1,date;
                """,

}


In [41]:
## SCHOOLS ##
all_params['infrastructure']['schools']['charts']['PPE'] = {
    'title' : """
                ***Schools***   
                Per Pupil Expenditures by school district
                [src](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx)
    
    """,
    'chart_type':'line',
    'metrics' : [ 'value' ] ,
    'XY'      : ( 'year' , 'item' ),
    'color'   : 'ppx',
    'traces'   : ['ppx'] ,
    'traces_visible'  : [ 'In-District Expenditures per Pupil' ],
    'query' :"""
                select x.school_id, x.year,ppx, x.value
                from infrastructure.schools_ppx x
                left join common.school_id_location lo on lo.school_id=x.school_id
                where lo.dor = 10
                order by ppx, x.year
                ;
            """,
}


##NEEDS WORK
all_params['infrastructure']['schools']['charts']['mcas'] = {
    'title' : """
                ***Schools***   
                MCAS perfomace by school, subject and grade
    
    """,
    'chart_type':'line',
    
    'metrics' : [
        "M+E %",
        "NM %",
        "No. of Students Included",
        "Part. Rate %",
        "Avg. Scaled Score",
        "Avg. SGP",
        "Median SGP",
        "Ach. PCTL"
    ] ,
    'XY'      : ( 'year' , 'item' ),
    'color'   : 'school_id',
    'traces'   : ['school_id','mcas_grade','mcas_subject'] ,
    'trace'   : ['school_id'] ,
    'traces_visible'  : [ '06','AL','ARLINGTON - HARDY','ARLINGTON - BRACKETT' ],
    'query' :"""
                SELECT x.*
                from infrastructure.schools_mcas x
                left join common.school_id_location lo on lo.school_id=x.school_id
                where lo.dor = 10
                and mcas_subject=2
                and mcas in (1,9,10,11,12,13,17,19)
                ;
            """,
}

all_params['infrastructure']['schools']['charts']['grades'] = {
    'title' : """
                ***Schools***    
                Enrollment by school and grade
                [src](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx)
    
    """,
    'chart_type':'line',    
    'metrics' : [ 'students' ] ,
    'XY'      : ( 'year' , 'students' ),
    'color'   : 'grade',
    'traces'   : ['school_id','grade'] ,
    'trace'   : ['grade'] ,
    'traces_visible'  : [ 'K','1','ARLINGTON - HARDY','ARLINGTON - BRACKETT' ],
    'query' :"""
                SELECT x.{item},x.year,sum(x.value) as students
                from infrastructure.schools_enrollment x
                left join common.school_id_location lo on lo.school_id=x.school_id
                where lo.dor = 10
                and x.school_id!=100000
                and x.grade!='Total'
                group by x.{item}, x.year
                order by x.{item}, x.year
                ;
            """,
}

all_params['infrastructure']['schools']['charts']['enrollment'] = {
    'title' : """
                ***Schools***   
                Enrollment by school type and year
                [src](https://profiles.doe.mass.edu/statereport/enrollmentbygrade.aspx)
    
    """,
    'chart_type':'line',
    'metrics' : [ 'students' ] ,
    'XY'      : ( 'year' , 'item' ),
    'color'   : 'schools_type',
    'traces'   : ['schools_type'] ,
    'traces_visible'  : [ 'Local Public Schools' ],
    'query' :"""
                select schools_type,year,value as students 
                from infrastructure.schools_types
                where dor=10
                order by schools_type, year desc
                ;
            """,
}

#### BOXPLOT

In [42]:
## DEEDS
all_params['property']['deeds']['charts']['box'] = {
    'title' : """
                ***REGISTERY OF DEEDS***    
                Boxplot of {count} documents in {date} totalling {total}
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
    """,
    'chart_type' : 'boxplot',
    'x'     : 'year' ,
    'y'     : 'consideration' ,
    'color' : 'deed_type',
    'traces' : ['deed_type'],
    'y_range' : [int(0),int(4e6)],
    'hover_data' :['date','deed_type','book','page','consideration',
                   'streetname','streetnum','unit'],
    'traces_visible'     : ["MORTGAGE","DEED"],
    'query'  :  """
                   SELECT cast(date_part('year', date) as int) as year, 
                           deed_type, date, book,page, consideration,
                          "streetname", "streetnum", unit
                   FROM property.deeds_details
                   WHERE cast(date_part('year', date) as int) = {year}
                   AND   consideration>100

                   ;
                """,
    'query_dates' : """
                        SELECT distinct cast(date_part('year', date) as int) as dates
                        FROM   property.deeds_details
                        ORDER BY dates;
                    """,                
    
}

In [43]:
## parcels
all_params['property']['parcels']['charts']['box'] = {
    'title' : """
                ***{ppt} - {dataset} from MassGIS***    
                Boxplot of {count} parcels in {date} totalling {total}
                [src](https://www.mass.gov/info-details/massgis-data-property-tax-parcels)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/parcels%20ETL.ipynb)
    
    """,
    'chart_type' : 'boxplot',
    'x'     : 'year' ,
    'y'     : 'total' ,
    'color' : 'land_use',
    'traces' : ['land_use'],
    'y_range' : [int(0),int(3e6)],
    'traces_visible'     : ["101 - Single Family","Ranch"],
    'hover_data' : ['year','total','building','land','owner','streetname','streetnum','unit'],
    'query'  :  """
                   SELECT year , land_use , land, building, total, owner, streetname, streetnum, unit 
                   FROM property.assessments
                   WHERE year = {year}
                   ;
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   property.assessments
                        ORDER BY dates;
                    """,                
    
}


In [44]:
## sales

all_params['property']['sales']['charts']['box'] = {
    'title' : """
                ***Real Estate Property Sales - Assessors***   
                Boxplot of {count} sales in {date} by land use
                [src](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales%20ETL.ipynb)
    
    """,
    'chart_type' : 'boxplot',    
    'x'     : 'year' ,
    'y'     : 'price' ,
    'color' : 'land_use',
    'traces' : ['land_use'],
    'y_range' : [int(0),int(4e6)],
    'traces_visible'     : ["101 - Single Family","102 - Condominium", "104 - Two Family", "Arms Length","Cape","Colonial","Ranch"],
    'hover_data' : ['price','date','buyer','seller','streetName','streetNum','unit'],
    'query'  :  """
                   SELECT year , sale_type , date , land_use , price, buyer, seller, "streetName", "streetNum", unit 
                   FROM property.sales
                   WHERE year = {year}
                   ;
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   property.sales
                        ORDER BY dates;
                    """,                
    
}


In [45]:
## permits
all_params['property']['permits']['charts']['box'] = {
    'title' : """
                ***{ppt} - {dataset} ***   
                Boxplot of {count} permits in {date} by permit type
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits%20ETL.ipynb)
    
    """,
    'chart_type' : 'boxplot',    
    'x'     : 'year' ,
    'y'     : 'permit_value' ,
    'color' : 'permit_type',
    'traces' : ['permit_type'],
    'y_range' : [int(0),int(1e6)],
    'traces_visible'     : ["Building"],
    'hover_data' : ['date','permit','description','owner',
                    'contractor','streetName','streetNum','pid'],
    'query'  :  """
                   SELECT * 
                   FROM property.permits
                   WHERE year = {year}
                   ;
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   property.permits
                        ORDER BY dates;
                    """,                
    
}


In [46]:
## water

all_params['infrastructure']['water']['charts']['box'] = {
    'title' : """
                ***Infrastructure***   
                Boxplot of {count} water bills on {date}
                [src](https://www.arlingtonma.gov/departments/public-works)
    """,
    'chart_type' : 'boxplot',
    'x'     : 'date' ,
    'y'     : 'amount' ,
    'color' : 'date',
    'traces' : ['date'],
    'y_range' : [int(0),int(5000)],
    #'traces_visible'     : ["2022-08-31","2022-11-30","2023-02-28"],
    'hover_data' : ['date','usage','amount','owner','address'],
    'query'  :  """
                   SELECT b.date,b.usage,b.amount, a.owner,a.address
                   FROM infrastructure.water_bills b
                   LEFT JOIN infrastructure.water_accounts a ON a.account = b.account
                   AND a.date = (
                       SELECT MIN( z.date ) 
                       FROM infrastructure.water_accounts z 
                       WHERE z.account = b.account AND z.date >= b.date
                    )
                   WHERE b.date = '{date}'
                   ;
                """,
    'query_dates' : """
                        SELECT distinct(date) as dates 
                        FROM infrastructure.water_bills 
                        ORDER BY dates;""",

}

#### HISTOGRAMS

In [47]:
## PEOPLE
##people again, no attributes
all_params['people']['elections']['charts']['histogram'] = {
    'title' : """
                ***Elections***   
                Age Histogram of Voters for the past {groups} elections
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    """,
    'chart_type' : 'histogram',
    'traces' : [
        'Local',
        'Mid-term',
        'Presidential',
        'State Primary',
        'Special State',
        'Presidential Primary',
        'Override'],
    'x'     : 'age' ,
    'y'     : 'count',
    'color' : 'date',
    'barmode'    : 'overlay',
    'range_x'    : [int(16),int(100)],
    'query'  :  """
                   SELECT * 
                   FROM people.activity_type_histogram
                   WHERE activity_type=0
                   and date in 
                   (select date from common.election_dates ed where ed.value='{item}')
                   ;
                """,
    
}

all_params['people']['elections']['charts']['groups'] = {
    'title' : """
                ***Elections***   
                Age Histogram of Voters for the {date} election over {groups} groups
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    """,
    'chart_type' : 'histogram',
    'traces' : [
        'Local',
        'Mid-term',
        'Presidential',
        'State Primary',
        'Special State',
        'Presidential Primary',
        'Override'],
    'traces_visible':['Male','Female'],
    'x'     : 'age' ,
    'y'     : 'count',
    'color' : 'value',
    'barmode'    : 'overlay',
    'range_x'    : [int(16),int(100)],
    'query'  :  """
                    SELECT date,age,count,
                    concat('P',cast(precinct as varchar)) as value
                    FROM people.precinct_histogram
                    WHERE date = '{date}' 
                    
                    UNION
                    
                    SELECT date,age,count,
                    ivp.value as value
                    FROM people.party_histogram h
                    LEFT JOIN common.int_value_pairs ivp 
                        ON ivp.key=h.party and ivp.item='party'
                    WHERE date = '{date}' 
                    
                    UNION
                    
                    SELECT date,age,count,
                    ivp.value as value
                    FROM people.sex_histogram h
                    LEFT JOIN common.int_value_pairs ivp 
                        ON ivp.key=h.sex and ivp.item='sex'
                    WHERE date = '{date}' 
                   ;
                """,
    'query_dates' : """
                SELECT    date  AS dates 
                FROM      common.election_dates 
                WHERE     value='{item}' 
                ORDER BY  dates;
            """,
    
}


all_params['people']['registered']['charts']['histogram'] = {
    'title' : """
                ***People - Voter Rolls***   
                Age Histogram of Registered Voters recorded on {groups} dates
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://www.mass.gov/info-details/mass-general-laws-c51-ss-47c)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    
    """,
    'chart_type' : 'histogram',
    'x'     : 'age' ,
    'y'     : 'count',
    'color' : 'date',
    'barmode'    : 'overlay',
    'range_x'    : [int(16),int(100)],
    'traces':['date'],
    'traces_visible'     : ['registered'],
    'query'  :  """
                   SELECT * 
                   FROM people.activity_type_histogram
                   WHERE activity_type=1
                   ;
                """,
    
}

all_params['people']['registered']['charts']['groups'] = {
    'title' : """
                ***People - Voter Rolls***   
                Age Histogram of Registered Voters as of {date} over {groups} groups
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://www.mass.gov/info-details/mass-general-laws-c51-ss-47c)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
    
    """,
    'traces':['value'],
    'traces_visible':['Democrat','Unenrolled'],
    'chart_type' : 'histogram',
    'x'     : 'age' ,
    'y'     : 'count',
    'color' : 'value',
    'barmode'    : 'overlay',
    'range_x'    : [int(16),int(100)],
    'query'  :  """
                    SELECT date,age,count,
                    concat('P',cast(precinct as varchar)) as value
                    FROM people.precinct_histogram
                    WHERE date = '{date}' 
                    
                    UNION
                    
                    SELECT date,age,count,
                    ivp.value as value
                    FROM people.party_histogram h
                    LEFT JOIN common.int_value_pairs ivp 
                        ON ivp.key=h.party and ivp.item='party'
                    WHERE date = '{date}' 
                    
                    UNION
                    
                    SELECT date,age,count,
                    ivp.value as value
                    FROM people.sex_histogram h
                    LEFT JOIN common.int_value_pairs ivp 
                        ON ivp.key=h.sex and ivp.item='sex'
                    WHERE date = '{date}' 
                   ;
                """,
    'query_dates' : """
                    SELECT distinct date as dates
                    FROM   people.activity_type_histogram
                    WHERE activity_type=1
                    ORDER BY dates;
                """,
    
}

all_params['people']['residents']['charts']['histogram'] = {
    'title' : """
                ***People - Residents***   
                Age Histogram of Residents recorded on {groups} dates
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://malegislature.gov/Laws/GeneralLaws/PartI/TitleVIII/Chapter51/Section6)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
                [4](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
    """,
    'chart_type' : 'histogram',
    'x'     : 'age' ,
    'y'     : 'count',
    'color' : 'date',
    'barmode'    : 'overlay',
    'range_x'    : [int(16),int(100)],
    'traces_visible'     : ['residents'],
    'query'  :  """
                   SELECT * 
                   FROM people.activity_type_histogram
                   WHERE activity_type=2
                   ;
                """,
}

all_params['people']['residents']['charts']['groups'] = {
    'title' : """
                ***People - Residents***   
                Age Histogram of Residents as of {date} over {groups} groups
                [src](https://www.arlingtonma.gov/town-governance/elections-voting)
                [1](https://malegislature.gov/Laws/GeneralLaws/PartI/TitleVIII/Chapter51/Section6)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/people/people.md)
                [3](https://github.com/irl-labs/polis/blob/main/notebooks/people/people%20ETL.ipynb)
                [4](https://medium.com/@irl-labs/the-people-of-arlington-ma-8af33002b55)
    """,
    'chart_type' : 'histogram',
    'traces':['value'],
    'traces_visible':['P3','P12'],
    'x'     : 'age' ,
    'y'     : 'count',
    'color' : 'value',
    'barmode'    : 'overlay',
    'range_x'    : [int(16),int(100)],
    'query'  :  """
                    SELECT date,age,count,
                    concat('P',cast(precinct as varchar)) as value
                    FROM people.precinct_histogram
                    WHERE date = '{date}' 
                    
                    UNION
                    
                    SELECT date,age,count,
                    ivp.value as value
                    FROM people.party_histogram h
                    LEFT JOIN common.int_value_pairs ivp 
                        ON ivp.key=h.party and ivp.item='party'
                    WHERE date = '{date}' 
                    
                    UNION
                    
                    SELECT date,age,count,
                    ivp.value as value
                    FROM people.sex_histogram h
                    LEFT JOIN common.int_value_pairs ivp 
                        ON ivp.key=h.sex and ivp.item='sex'
                    WHERE date = '{date}' 
                   ;
                """,
    'query_dates' : """
                    SELECT distinct date as dates
                    FROM   people.activity_type_histogram
                    WHERE activity_type=2
                    ORDER BY dates;
                """,
    
}

##### PROPERTY

In [48]:
## DEEDS
all_params['property']['deeds']['charts']['histogram'] = {
    'title' : """
                ***REGISTERY OF DEEDS***    
                Histogram of Sales and Mortgages in {groups} {trace} groups for {date}
                [src](http://www.masslandrecords.com/MiddlesexSouth)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/deeds%20ETL.ipynb)
    """,
    'chart_type':'histogram',
    'x'     : 'consideration' ,
    'color' : 'deed_type',
    'traces' : ['deed_type'],
    'barmode'    : 'overlay',
    'range_x'    : [int(100000),int(3000000)],
    'traces_visible'     : ["DEED","MORTGAGE"],
    'query'  :  """
                   SELECT cast(date_part('year', date) as int) as year , 
                           deed_type , consideration 
                   FROM property.deeds_details
                   WHERE cast(date_part('year', date) as int) = {year}
                   AND   consideration>100
                   ;
                """,
    'query_dates' : """
                        SELECT distinct cast(date_part('year', date) as int) as dates
                        FROM   property.deeds_details
                        ORDER BY dates;
                    """,
    
}

In [49]:
## parcels

all_params['property']['parcels']['charts']['histogram'] = {
    'title'   : """
        ***Property***:  Histogram of {metric} assessments in {groups} {trace} groups for {date}.
        """,
    'source'  : """[source](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)""",
    'discuss' : """""",#    'title'   : 'Assesssments Histogram by Use'            ,
    'chart_type':'histogram',
    'traces' : ['land_use','style','zoning','decade_built'] ,
    'x'     : 'total' ,
    'color' : 'land_use',
    'metrics':['total','land','building','total_chg_1y','land_chg_1y'],
    'barmode'    : 'overlay',
    'range_x'    : [int(200000),int(2000000)],
    'range2_x'   : [int(-5),int(40)],
    'traces_visible'     : ["101 - Single Family","102 - Condominium", "Arms Length","Cape","Colonial","Ranch", "R1", "R2","1920","1930","1950"],
    'query'  :  """
                   SELECT a.year , land_use , style, zoning,
                        cast(10*date_part('decade',to_date(year_built::varchar, 'yyyy')) as int)::text as decade_built,
                        total , 
                        building , 
                        land , 
                        ROUND ( 100 * cast ( total_chg_1y as numeric ) , 2 ) as total_chg_1y,
                        ROUND ( 100 * cast ( land_chg_1y  as numeric ) , 2 ) as land_chg_1y
                    FROM property.assessments a
                    WHERE a.year = {year}
                   ;
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   property.assessments
                        ORDER BY dates;
                    """,                
    
}




In [50]:
## sales
all_params['property']['sales']['charts']['histogram'] = {
    'title' : """
                ***Real Estate Property Sales - Assessors***   
                Histogram of sales in {groups} {trace} groups for {date}
                [src](https://dlsgateway.dor.state.ma.us/gateway/DLSPublic/ParcelSearch)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/sales%20ETL.ipynb)
    
    """,
    'chart_type':'histogram',
    'traces' : ['land_use','sale_type'] ,
    'x'     : 'price' ,
    'color' : 'land_use',
    'barmode'    : 'overlay',
    'range_x'    : [int(100000),int(3000000)],
    'traces_visible'     : ["101 - Single Family","Arms Length","Substantial Change"],
    'query'  :  """
                   SELECT year , sale_type , date , land_use , price 
                   FROM property.sales s
                   WHERE year = {year}
                   ;
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   property.sales
                        ORDER BY dates;
                    """,
    
}


In [51]:
##permits
all_params['property']['permits']['charts']['histogram'] = {
    'title' : """
                ***{ppt} - {dataset} ***   
                Histogram of permits in {groups} {trace} groups for {date}
                [src](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits.md)
                [2](https://github.com/irl-labs/polis/blob/main/notebooks/property/permits%20ETL.ipynb)
    
    """,
    'chart_type':'histogram',
    'x'     : 'permit_value' ,
    'color' : 'permit_type',
    'traces':['permit_type'],
    'barmode'    : 'overlay',
    'range_x'    : [int(0),int(100000)],
    'traces_visible'     : ['Building','Residential'],
    'query'  :  """
                   SELECT year , permit_type , permit_value 
                   FROM property.permits
                   WHERE year = {year}
                   ;
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   property.permits
                        ORDER BY dates;
                    """,
    
}

In [52]:
## ENERGY ##
all_params['property']['energy']['charts']['histogram'] = {
    'title' : """
                ***Solar Energy***   
                Histogram of solar installs in {groups} {trace} groups for {date}
                [src](https://www.masscec-pts.com/#/home)
                [1](https://seigniorage.medium.com/mapping-the-solar-system-61ba53fd23aa)
                [2](https://emp.lbl.gov/tracking-the-sun) 
                [3](https://www.masscec.com/sites/default/files/documents/PVinPTSwebsite%202022-02-22.xlsx)
                [4](https://www.arlingtonma.gov/departments/inspectional-services/view-building-permits)
    """,
    'chart_type':'histogram',
    'traces' : ['year','pts_installer',
                'pts_module_mfgr',
                'pts_inverter_mfgr'] ,
    'metrics' : ['cost_per_watt','kw_installed'] ,
    'x'     : 'cost_per_watt' ,
    'color' : 'year',
    'barmode'    : 'overlay',
    'range_x'    : [int(2),int(10)],
    'traces_visible'     : ['2020','2021','Sunrun Inc.','Trinity Solar',
                            'LG Electronics','SunPower',
                           'Enphase Energy'],
    'query':"""
                select year,pts_installer,pts_module_mfgr,
                pts_inverter_mfgr,
                    cost/(watt*1.0) as cost_per_watt,
                	(watt/1000.0) as kw_installed 
                from energy.solar_pts 
                where year={date}
                order by date desc;
            """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   energy.solar_pts
                        ORDER BY dates;
                    """,
    
}


In [53]:
## financials

all_params['governance']['financials']['charts']['DOR histogram'] = {
    'title' : """
                ***Financials - Department of Revenue Databank***   
                {groups} Histograms in {series} for all municipalities - FY{date}
                [src](https://www.mass.gov/lists/schedule-a-reports-revenues-expenditures-and-more)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/governance/dor_databank.md)
    """,
    'chart_type':'histogram',
    'x'     : 'zscore' ,
    'color' : 'dor_databank_series',
    'traces' : ['dor_databank_series'],
    #'nbins' : int(100),
    #'xbins_size' :  0.05 ,
    'barmode'    : 'overlay',
    'range_x'    : [-int(2),int(2)],
    'traces_visible'  : FINANCIALS_VISIBLE_TRACES,
    'query'   : """
                    SELECT           f.dor, f.year,f.value as value,f.zscore,
                                     f.dor_databank_series
                    FROM             governance.dor_databank f 
                    WHERE            f.year={year}
                    AND              f.year >= 1990
                    AND  f.dor_databank_series in (
                        select ivp.key
                        from common.int_value_pairs ivp
                        LEFT JOIN common.int_value_pairs ivp2
                            ON ivp2.item = 'dor_databank_series_types' 
                            AND ivp2.value='{series_type}'
                        LEFT JOIN common.int_value_pairs ivp3
                            ON ivp3.item = 'dor_databank_series_type' 
                            AND ivp3.value=ivp2.value
                        WHERE ivp.item='dor_databank_series'
                        and ivp.key=ivp3.key
                        and ivp3.key is not NULL
                    )
                    ORDER BY dor_databank_series, year
                """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   governance.dor_databank
                        WHERE  year >= 1990 and year<=2023
                        ORDER BY dates;
                    """,
    'query_series' : """SELECT key,value FROM common.int_value_pairs WHERE item='dor_databank_series_types';""",
    'annotate'      :  'ArlingtonMA',
    
}


In [54]:
##water
all_params['infrastructure']['water']['charts']['histogram'] = {
    'title' : """
                ***Infrastructure***   
                Histogram of water usage for {date}
                [src](https://www.arlingtonma.gov/departments/public-works)
    """,
    'chart_type':'histogram',
    'metrics' : ['amount','usage',
                 'amount_ttm','usage_ttm',
                 'amount_chg_YoY','usage_chg_YoY'] ,
    'x'     : 'amount' ,
    'color' : 'date',
    'traces':['date'],
    'barmode'    : 'overlay',
    'range_x'    : [int(0),int(3000)],
    'range2_x'   : [int(-1),int(2)],
    'traces_visible'     :  ['2023-02-28','2022-11-30','2022-08-31','2022-05-31','2019-05-31'],
    'query'  :  """
                   SELECT date , amount, usage,
                   amount_ttm,usage_ttm,
                "amount_chg_YoY","usage_chg_YoY"
                   FROM infrastructure.water_bills
                   WHERE date = '{date}'
                   ;
                """,
    'query_dates' : """
                        SELECT distinct(date) as dates 
                        FROM infrastructure.water_bills 
                        ORDER BY dates;""",
}

In [55]:
## ROADS ##
all_params['infrastructure']['roads']['charts']['histogram'] = {
    'title' : """
                ***Roads and Intersections***   
                Histogram of {town} road lengths
                [src](https://gis-arlingtonma.opendata.arcgis.com/maps/ed93de524bbe48f1a65e454cdbf7ef97)
                [1](https://www.arlingtonma.gov/home/showdocument?id=2468)
    
    """,
    'chart_type':'histogram',
    'metrics' : ['length','miles', 'left_sidewalk', 'right_sidewalk'] ,
    'x'     : 'length' ,
    'color' : 'class',
    'traces':[ 'class'],
    'barmode'    : 'overlay',
    'range_x'    : [int(0),int(1500)],
    'range2_x'    : [int(1),int(30)],
    #'traces_visible'     :  ["Local Street","Private Way"],
    'query'  :  """
                   SELECT class, length, miles, left_sidewalk, right_sidewalk 
                   FROM infrastructure.roads
                   ;
                """

}

In [56]:
## GRID ##
all_params['infrastructure']['grid']['charts']['histogram'] = {
    'title' : """
                ***Eversource Electric Grid***   
                Histogram of electrical grid transformer {metric} 
                [src](https://eversource.maps.arcgis.com/apps/webappviewer/index.html?id=7b13d31f908243e49406f198b359aa71)
    
    """,
    'chart_type':'histogram',
    'metrics' : ['capacity_MW','amps'] ,
    'x'     : 'capacity_MW' ,
    'color' : 'circuit_id',
    'traces': ['circuit_id'],
    'barmode'    : 'overlay',
    'range_x'    : [int(0),int(3)],
    'range2_x'    : [int(100),int(500)],
    #'traces_visible'     :  ["211-H14","320-H7"],
    'query'  :  """
                   SELECT circuit_id, "capacity_MW", amps 
                   FROM infrastructure.grid
                   ;
                """
}

#### SCATTER

In [57]:
## schools scatter
all_params['infrastructure']['schools']['charts']['scatter'] = {
    'title' : """
                ***Schools***: Correlation chart of paired MCAS and financial
                 metrics for all municipalities (timeseries on hover).
    """,
    'source'  : """[Source]()""",
    'discuss' : """""",
    'query':"""
                select m.school_id,m.year,ivp2.value as category,m.value
                from infrastructure.schools_mcas m
                left join common.int_value_pairs ivp 
                    on ivp.key = m.school_id and ivp.item='district'
                left join common.int_value_pairs ivp2 
                    on ivp2.key = m.mcas and ivp2.item='mcas'
                where mcas_grade=7
                and ivp.key is not null and ivp2.key is not null
                and mcas_subject=2

                UNION

                select m.school_id,m.year,ivp2.value as category,m.value
                from infrastructure.schools_ppx m
                left join common.int_value_pairs ivp 
                    on ivp.key = m.school_id and ivp.item='district'
                left join common.int_value_pairs ivp2 
                    on ivp2.key = m.ppx and ivp2.item='ppx'
                where ivp.key is not null and ivp2.key is not null

                order by school_id, year, category
    """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   infrastructure.schools_mcas
                        ORDER BY dates;
                    """,                    
    
}


In [58]:
## FINANCIALS scatter ##


all_params['governance']['financials']['charts']['DOR scatter'] = {
    'title' : """
                ***Financials - Department of Revenue Databank***   
                Correlation chart of paired financial metrics in {date} for all municipalities (timeseries on hover)
                [src](https://www.mass.gov/lists/schedule-a-reports-revenues-expenditures-and-more)
                [1](https://github.com/irl-labs/polis/blob/main/notebooks/governance/dor_databank.md)
    """,
    'chart_type':'scatter',
    'query':"""
        SELECT           f.dor, f.year,f.value as value,f.zscore,
                         f.dor_databank_series
        FROM             governance.dor_databank f 
        WHERE            f.year >= 1990 
        AND              f.year <= 2023
        AND              f.dor_databank_series in (0, 3, 4, 11, 13, 14, 16, 17, 23, 24, 34, 
                                                    187, 188, 223, 224, 241, 243, 261, 262, 267, 269,
                                                    315, 317, 318, 319, 320, 321, 328, 352, 355, 379, 
                                                    380, 384, 391, 396, 402, 417, 429, 435, 437, 463, 474)
        ORDER BY dor, year
        ;
    """,
    'query_dates' : """
                        SELECT distinct year as dates
                        FROM   governance.dor_databank
                        WHERE  year >= 1990 and year <= 2023
                        ORDER BY dates;
                    """,                    
    
}


### LOADER

In [59]:
import pandas as pd

town='ArlingtonMA'
query = "select town,town_id,sum_acres as acres,sum_square as miles, geometry,lat,lon from common.town_boundaries where upper(town)='{town}'"\
    .format(town=town[0:-2].upper())
town_boundaries = pd.read_sql_query(query,cnx)

lat=town_boundaries.lat.values[0]
lon=town_boundaries.lon.values[0]
print(f'lat={lat} lon={lon}')


##series_type for financials
query = "select key,value from common.int_value_pairs where item='dor_databank_series_types';"
loptions = { y : x for x, y in pd.read_sql(query,con=cnx).to_dict()['value'].items()}

for view in ['charts','maps','tables']:
    for key in all_params['governance']['financials'][view].keys(): 
        if view=='charts':
            if key!='scatter':
                all_params['governance']['financials'][view][key]['series']=list(loptions.keys())
        else:
            if key not in ['checkbook','prop 2.5']:
                all_params['governance']['financials'][view][key]['series']=list(loptions.keys())


all_params = ddict2dict(all_params)

for ppt in all_params.keys():  #people, property
    for attribute in all_params[ppt].keys(): ##sales, deeds
        for view in all_params[ppt][attribute].keys():  ##map, table, chart
            for display in all_params[ppt][attribute][view].keys(): ##scatter, line, detail
                print(ppt,attribute,view,display)
                if view == 'maps':
                    if display != 'state':
                        all_params[ppt][attribute][view][display]['lat']=lat
                        all_params[ppt][attribute][view][display]['lon']=lon
                if 'query_dates' in all_params[ppt][attribute][view][display]:
                    query = all_params[ppt][attribute][view][display]['query_dates'].format(item='Local')
                    dates = pd.read_sql(query,con=cnx)['dates'].to_list()
                    dates = [ x if type(x)==int else x.strftime('%Y-%m-%d') for x in dates ]
                    all_params[ppt][attribute][view][display]['dates']=dates




lat=42.418655 lon=-71.1639
property deeds tables detail
property deeds tables summary
property deeds tables summary by year and filing type
property deeds tables sales summary
property deeds maps detail
property deeds maps sales
property deeds charts line
property deeds charts box
property deeds charts histogram
property energy tables BNL detail
property energy tables EV chargers
property energy tables PTS detail
property energy tables PTS summary
property energy tables PTS summary all
property energy tables heat pumps
property energy tables solar arrays
property energy tables solar contractors
property energy tables solar contractors - all
property energy tables systems
property energy tables solar production
property energy maps  Solar systems
property energy maps EV chargers
property energy maps Heat Pumps
property energy maps Heat Pumps 2
property energy maps Solar - Groups
property energy maps HVAC
property energy charts solar line
property energy charts histogram
property parcels

In [60]:
import pandas as pd
from sqlalchemy import dialects

table_create_query = \
    """
        DROP TABLE IF EXISTS common.graphics_parameters;
        CREATE TABLE common.graphics_parameters (
            "parameters"  JSONB
        )
       ;
    """

cnx.execute(table_create_query)
df = pd.DataFrame({'parameters':[all_params]})

df.to_sql (
        'graphics_parameters',
        schema='common',
        con=cnx,
        if_exists='append',
        index=False,
        dtype={'parameters': dialects.postgresql.JSONB}
)

/tmp/ipykernel_2476187/3364362436.py:13: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  cnx.execute(table_create_query)


1

In [61]:
import pandas as pd

query = "select * from common.graphics_parameters;"
all_params = pd.read_sql_query (query,con=cnx).iloc[0]['parameters']

cnx.execute("delete from common.parameters where parameter_type=1;")
df = pd.DataFrame([{"parameter_type":1,"dict":all_params}])
from sqlalchemy import dialects
df.to_sql(
    'parameters',schema='common',con=cnx,
    if_exists='append',index=False,
    dtype={'dict': dialects.postgresql.JSONB, }
)


1